**重要：delta_GPA绝对不允许对本文件做任何改动**

In [ ]:
'''
这个Jupyter Notebook包含了基于图神经网络（Graph Neural Network，GNN）的用于预测自由能（Gibbs Free Energy，ΔG）的函数。
如果用户请求使用GNN方法，对于简单的情形下的生化反应热力学分析，可以直接调用这些函数进行预测。

分为两大类：
- 生成自由能预测（Formation Energy，$Δ_f G$）
- 反应自由能预测（Reaction Energy，$Δ_r G$）
这些函数可以帮助用户在不同条件下预测自由能，从而支持生化反应的热力学分析
'''

In [1]:
'''
必要库，在调用本文档的函数之前请确保import这些库
'''
import subprocess
import json
import sys
import os
from typing import Optional, Tuple, Union, List

import numpy as np
import numpy.typing as npt
import pandas as pd
from rdkit import Chem

from equilibrator_api import ComponentContribution, Q_

In [2]:
from rdkit import RDLogger
import warnings

# 抑制报错输出
RDLogger.DisableLog('rdApp.*')
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [3]:
'''
初始化CC类，以便在后续函数中调用
'''
cc = ComponentContribution() # 初始化组分贡献法对象放在全局，避免重复初始化

## 生成自由能预测（Formation Energy，$Δ_f G$）

这部分包含了用于计算**化合物生成自由能**的函数

In [4]:
'''
必要函数，使用GNN模型进行预测
'''
from model.GNN_dGf import predict_standard_dGf_prime

## 已知化合物计算

In [5]:
def get_compound(identifier: str, cc) -> Optional[object]:
    """
    根据标识符获取化合物对象，按优先级尝试多种策略

    注意：调用本函数必须全局初始化cc = ComponentContribution()
    
    Args:
        identifier: 化合物标识符(支持InChI、KEGG、BIGG、Metacyc、SMILES等格式)
        - 格式1: InChI (以 "InChI=" 开头)
        - 格式2: KEGG (以 "C" 开头，后跟5位数字)
        - 格式3: BIGG (直接使用BIGG ID)
        - 格式4: Metacyc (直接使用Metacyc ID)
        - 格式5: SMILES (有效的SMILES字符串，最后尝试)
        cc: 已初始化的 ComponentContribution 对象
    
    Returns:
        成功时返回化合物对象，失败时返回 None
    """

    RDLogger.DisableLog('rdApp.error')  # 禁用RDKit错误日志输出

    def try_get_compound(query: str) -> Optional[object]:
        """尝试获取化合物，失败或返回None时返回None"""
        try:
            result = cc.get_compound(query)
            return result if result is not None else None
        except Exception:
            return None
    
    def is_smiles(s: str) -> bool:
        """判断字符串是否为有效的SMILES格式"""
        try:
            mol = Chem.MolFromSmiles(s)
            return mol is not None
        except Exception:
            return False
    
    def smiles_to_inchi(smiles: str) -> Optional[str]:
        """将SMILES转换为InChI"""
        try:
            mol = Chem.MolFromSmiles(smiles)
            if mol is not None:
                inchi = Chem.MolToInchi(mol)
                return inchi
            return None
        except Exception:
            return None
    
    # 策略0: 直接查询（原始标识符）
    compound = try_get_compound(identifier)
    if compound is not None:
        return compound
    
    # 策略0.5: 通用搜索
    compound = cc.search_compound(identifier)
    if compound is not None:
        return compound

    
    # 策略1: InChI格式
    compound = cc.get_compound_by_inchi(identifier)
    if compound is not None:
        return compound


    # 策略2: KEGG格式 (C + 5位数字)
    compound = try_get_compound(f"kegg:{identifier}")
    if compound is not None:
        return compound
    
    # 策略3: BIGG数据库
    compound = try_get_compound(f"bigg.metabolite:{identifier}")
    if compound is not None:
        return compound
    
    # 策略4: MetaCyc数据库
    compound = try_get_compound(f"metacyc.compound:{identifier}")
    if compound is not None:
        return compound
    
    # 策略5: SMILES格式（最后尝试，因为检测成本较高）
    if is_smiles(identifier):
        try:
            inchi = smiles_to_inchi(identifier)
            if inchi:
                compound = cc.get_compound_by_inchi(inchi)
                if compound is not None:
                    return compound
        except Exception:
            pass
    
    # 所有策略均失败
    return None

In [6]:
# 示例调用
get_compound("phophate", cc)

Compound(id=12, inchi_key=NBIIXXVUZAFLBC-UHFFFAOYSA-L)

In [7]:
def cc_transformed_standard_dgf_prime_GNN(
    input: str, 
    cc,
    p_h: float = 7.0, 
    p_mg: float = 3.0, 
    I: float = 0.25, 
    T: float = 298.15
) -> Tuple[np.floating, np.floating]:
    '''
    使用图神经网络(GNN)计算已知化合物的标准生成自由能（经过CC数据库的Legendre变换）
    
    注意：调用本函数必须先全局初始化cc = ComponentContribution()，同时调用 get_compound() 函数
    
    参数:
    input: 化合物的InChI字符串或其他Equilibrator API支持的格式
    p_h: 溶液的pH值 (默认值: 7.0)
    p_mg: 溶液的pMg值 (默认值: 3.0)
    I: 离子强度，单位为M (默认值: 0.25M)
    T: 温度，单位为K (默认值: 298.15K)
    
    返回:
    standard_dgf_prime_GNN: 物质在指定条件下的形成自由能 (Δ_fG'°, kJ/mol)
    std_GNN: 生成自由能误差 (kJ/mol)
    '''
    
    # 获取化合物
    cpd = get_compound(input, cc)
    if cpd is None:
        raise ValueError(f"无法找到化合物: {input}")

    # 步骤1: GNN 返回 未 transform 的自由能值
    standard_dgf_c_GNN, std_GNN = predict_standard_dGf_prime(cpd.inchi)
    
    # 步骤2: Legendre变换，转换到用户指定的条件
    delta_user_c = cpd.transform(p_h = Q_(7.0), p_mg = Q_(14.0), ionic_strength = Q_('0.25M'), temperature = Q_('298.15K'))
    standard_dgf_GNN = standard_dgf_c_GNN - delta_user_c.m_as("kJ/mol")

    delta_user = cpd.transform(p_h = Q_(p_h), p_mg = Q_(p_mg), ionic_strength = Q_(f'{I}M'), temperature = Q_(f'{T}K'))
    standard_dgf_prime_GNN = standard_dgf_GNN + delta_user.m_as("kJ/mol")

    return float(standard_dgf_prime_GNN), float(std_GNN)

In [8]:
# 示例调用
dGf, std = cc_transformed_standard_dgf_prime_GNN('glucose', cc) # 注意输入化合物名称时一定使用小写
print(f"Glucose Δ_fG'° (Standard conditions): {dGf:.2f} ± {std:.2f} kJ/mol")

Glucose Δ_fG'° (Standard conditions): -424.44 ± 0.60 kJ/mol


## 未知化合物计算

In [9]:
def convert_to_inchi(input_string):
    """
    输入字符串，如果是SMILES就转换成InChI，如果是InChI就直接保留
    
    参数:
        input_string: 输入的化学结构字符串
    
    返回:
        InChI格式的字符串
    """
    input_string = input_string.strip()
    
    # 判断是否为InChI格式（InChI以"InChI="开头）
    if input_string.startswith("InChI="):
        print("检测到InChI格式，直接保留")
        return input_string
    else:
        # 假设是SMILES格式，尝试转换为InChI
        print("检测到SMILES格式，转换为InChI")
        try:
            mol = Chem.MolFromSmiles(input_string, sanitize=False)
            if mol is not None:
                inchi = Chem.MolToInchi(mol)
                return inchi
            else:
                return "错误：无效的SMILES字符串"
        except Exception as e:
            return f"错误：转换失败 - {str(e)}"

In [10]:
def untransformed_standard_dgf_GNN(
    input: str, 
    cc,
    p_h: float = 7.0, 
    p_mg: float = 3.0, 
    I: float = 0.25, 
    T: float = 298.15
) -> Tuple[np.floating, np.floating]:
    '''
    使用图神经网络(GNN)计算全新化合物的生成自由能（缺少pKa信息，未经过Legendre变换）
    
    注意：调用本函数必须先全局初始化cc = ComponentContribution()，同时调用 convert_to_inchi() 函数
    
    参数:
    input: 化合物的InChI字符串或其他Equilibrator API支持的格式
    p_h: 溶液的pH值 (默认值: 7.0)
    p_mg: 溶液的pMg值 (默认值: 3.0)
    I: 离子强度，单位为M (默认值: 0.25M)
    T: 温度，单位为K (默认值: 298.15K)
    
    返回:
    standard_dgf_prime_GNN: 物质在指定条件下的形成自由能 (Δ_fG'°, kJ/mol)
    std_GNN: 生成自由能误差 (kJ/mol)
    '''

    cpd_inchi = convert_to_inchi(input)
    # GNN 返回 未 transform 的自由能值
    standard_dgf_GNN, std_GNN = predict_standard_dGf_prime(cpd_inchi)

    return float(standard_dgf_GNN), float(std_GNN)

In [19]:
# 示例调用
smiles = "CC(C)CCCCCC1=CC=C(C=C1)C(C)C(=O)O"
dgf, uncertainty = untransformed_standard_dgf_GNN(smiles, cc)
print(f"Method: {method}, ΔGf: {dgf:.2f} ± {uncertainty:.2f} kJ/mol")

检测到SMILES格式，转换为InChI
Method: unknown_untransformed, ΔGf: 883.53 ± 7.99 kJ/mol


## 自动判断已知/未知（推荐使用）

In [11]:
def calculate_standard_dgf_GNN(
    identifier: str,
    cc,
    p_h: float = 7.0,
    p_mg: float = 3.0,
    I: float = 0.25,
    T: float = 298.15
) -> Tuple[np.floating, np.floating, str]:
    """
    自动判断化合物是否已知，并选择合适的方法计算标准生成自由能
    
    工作流程:
    1. 尝试通过 get_compound() 查询化合物
    2. 如果找到(已知化合物)：使用 cc_transformed_standard_dgf_prime_GNN (经过Legendre变换)
    3. 如果未找到(未知化合物)：使用 untransformed_standard_dgf_GNN (未经Legendre变换)
    
    注意：调用本函数必须先全局初始化cc = ComponentContribution()，同时调用以下函数：
        get_compound()
        cc_transformed_standard_dgf_prime_GNN()
        convert_to_inchi()
        untransformed_standard_dgf_GNN()

    Args:
        identifier: 化合物标识符（支持InChI、KEGG、BIGG、Metacyc、SMILES等格式）
        cc: ComponentContribution 对象
        p_h: pH值，默认7.0
        p_mg: pMg值，默认3.0
        I: 离子强度，默认0.25 M
        T: 温度，默认298.15 K
    
    Returns:
        Tuple[np.floating, np.floating, str]: 
            - 标准生成自由能 (kJ/mol)
            - 不确定度 (kJ/mol)
            - 计算方法标识 ("known_transformed" 或 "unknown_untransformed")
    
    Example:
        >>> cc = ComponentContribution()
        >>> # 已知化合物
        >>> dgf, uncertainty, method = calculate_standard_dgf_GNN("KEGG:C00002", cc)
        >>> print(f"Method: {method}, ΔGf: {dgf:.2f} ± {uncertainty:.2f} kJ/mol")
        
        >>> # 未知化合物
        >>> smiles = "CC(C)CC1=CC=C(C=C1)C(C)C(=O)O"  # 布洛芬
        >>> dgf, uncertainty, method = calculate_standard_dgf_GNN(smiles, cc)
        >>> print(f"Method: {method}, ΔGf: {dgf:.2f} ± {uncertainty:.2f} kJ/mol")
    """
    # 尝试获取化合物对象
    compound = get_compound(identifier, cc)
    
    if compound is not None:
        # 检查化合物是否有有效的 InChI
        has_valid_inchi = hasattr(compound, 'inchi') and compound.inchi is not None and compound.inchi.strip() != ""
        
        if has_valid_inchi:
            # 已知化合物且有InChI：使用经过Legendre变换的方法（CC数据库）
            try:
                dgf, uncertainty = cc_transformed_standard_dgf_prime_GNN(
                    input=identifier,
                    cc=cc,
                    p_h=p_h,
                    p_mg=p_mg,
                    I=I,
                    T=T
                )
                method = "known_transformed"
                print(f"✓ 已知化合物：使用CC数据库的Legendre变换方法")
                return dgf, uncertainty, method
                
            except Exception as e:
                print(f"⚠ 已知化合物但计算失败: {str(e)}")
                print(f"  尝试作为未知化合物处理...")
                # 继续执行下面的未知化合物逻辑
        else:
            print(f"⚠ 化合物已找到但缺少InChI信息")
            print(f"  尝试作为未知化合物处理...")
    
    # 未知化合物或已知化合物但缺少InChI：使用未经Legendre变换的GNN预测方法
    print(f"✗ 未知化合物：使用GNN预测方法（未经Legendre变换，缺少pKa信息）")
    
    # 需要先转换为InChI格式
    inchi = convert_to_inchi(identifier)
    
    dgf, uncertainty = untransformed_standard_dgf_GNN(
        input=inchi,
        cc=cc,
        p_h=p_h,
        p_mg=p_mg,
        I=I,
        T=T
    )
    method = "unknown_untransformed"
    
    return dgf, uncertainty, method

In [12]:
# 示例调用
known_smiles = "C1=CC=CC=C1"  # 苯的SMILES
dgf, uncertainty, method = calculate_standard_dgf_GNN(known_smiles, cc)
print(f"Method: {method}, ΔGf: {dgf:.2f} ± {uncertainty:.2f} kJ/mol")

✓ 已知化合物：使用CC数据库的Legendre变换方法
Method: known_transformed, ΔGf: 322.31 ± 53.44 kJ/mol


In [13]:
# 示例调用
unknown_smiles = "CC(C)CC1=CC=C(C=C1)C(C)C(=O)O"  # 布洛芬的SMILES
dgf, uncertainty, method = calculate_standard_dgf_GNN(unknown_smiles, cc)
print(f"Method: {method}, ΔGf: {dgf:.2f} ± {uncertainty:.2f} kJ/mol")

⚠ 化合物已找到但缺少InChI信息
  尝试作为未知化合物处理...
✗ 未知化合物：使用GNN预测方法（未经Legendre变换，缺少pKa信息）
检测到SMILES格式，转换为InChI
检测到InChI格式，直接保留
Method: unknown_untransformed, ΔGf: 528.57 ± 7.64 kJ/mol


In [14]:
def calculate_standard_dgf_batch(
    identifiers: List[str],
    cc,
    p_h: float = 7.0,
    p_mg: float = 3.0,
    I: float = 0.25,
    T: float = 298.15
) -> pd.DataFrame:
    """
    批量计算多个化合物的标准生成自由能
    
    Args:
        identifiers: 化合物标识符列表
        cc: ComponentContribution 对象
        p_h: pH值，默认7.0
        p_mg: pMg值，默认3.0
        I: 离子强度，默认0.25 M
        T: 温度，默认298.15 K
    
    Returns:
        pd.DataFrame: 包含所有计算结果的数据框
            列: identifier, dgf, uncertainty, method, status
    
    Example:
        >>> cc = ComponentContribution()
        >>> compounds = ["KEGG:C00002", "KEGG:C00008", "CC(=O)O"]
        >>> results = calculate_standard_dgf_batch(compounds, cc)
        >>> print(results)
    """
    results = []
    
    for idx, identifier in enumerate(identifiers, 1):
        print(f"\n[{idx}/{len(identifiers)}] 处理: {identifier}")
        
        try:
            dgf, uncertainty, method = calculate_standard_dgf_GNN(
                identifier=identifier,
                cc=cc,
                p_h=p_h,
                p_mg=p_mg,
                I=I,
                T=T
            )
            
            results.append({
                'identifier': identifier,
                'dgf': float(dgf),
                'uncertainty': float(uncertainty),
                'method': method,
                'status': 'success'
            })
            
        except Exception as e:
            print(f"✗ 错误: {str(e)}")
            results.append({
                'identifier': identifier,
                'dgf': np.nan,
                'uncertainty': np.nan,
                'method': 'error',
                'status': str(e)
            })
    
    return pd.DataFrame(results)

In [15]:
# 示例调用
compounds = ["kegg:C00002", "kegg:C00008", "CC(=O)O"]
results = calculate_standard_dgf_batch(compounds, cc)
print(results)


[1/3] 处理: kegg:C00002
✓ 已知化合物：使用CC数据库的Legendre变换方法

[2/3] 处理: kegg:C00008
✓ 已知化合物：使用CC数据库的Legendre变换方法

[3/3] 处理: CC(=O)O
⚠ 化合物已找到但缺少InChI信息
  尝试作为未知化合物处理...
✗ 未知化合物：使用GNN预测方法（未经Legendre变换，缺少pKa信息）
检测到SMILES格式，转换为InChI
检测到InChI格式，直接保留
    identifier          dgf  uncertainty                 method   status
0  kegg:C00002 -2300.688195     1.195420      known_transformed  success
1  kegg:C00008 -1425.450491     1.021676      known_transformed  success
2      CC(=O)O  -246.664322     0.752392  unknown_untransformed  success


## 反应自由能预测（Reaction Energy，$Δ_r G$）

这部分包含了用于计算**生化反应自由能**的函数

注意：如果此部分函数运行失效，可以在生成自由能预测函数的基础上重新编写反应自由能预测函数，在工作区运行